In [1]:
import ee
import folium

Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it.

In [ ]:
ee.Authenticate()

#initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=mLD31CsVAixd8T3iE7LJ2TaIavA1kkNQrb5vQd62NA8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhu1S0pOlPon-FK7JgjvD6G53g3RnyjG_ugAoE4pedC50hC4FNZ1I0

Successfully saved authorization token.


Testing the API

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


getting Sentinel 1 image

In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              71.9121,
              -69.8065
            ],
            [
              78.3803,
              -69.8065
            ],
            [
              78.3803,
              -66.9398
            ],
            [
              71.9121,
              -66.9398
            ],
            [
              71.9121,
              -69.8065
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
center = [67.38929150741328, -66.84343082499258]
aoi = ee.Geometry.Point(center).buffer(25000)
S1A = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(ee.Date('2020-06-01'),ee.Date('2020-06-02'))

In [ ]:
S1A.size().getInfo()
# print(S1A.size())

1

In [ ]:
D28 = ee.Geometry.Rectangle([-71.01, -68.5, -73, -68])
D28Coordinates = D28.coordinates()
print(D28Coordinates)

ee.List({
  "functionInvocationValue": {
    "functionName": "Geometry.coordinates",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -71.01,
                    -68
                  ],
                  [
                    -71.01,
                    -68.5
                  ],
                  [
                    -73,
                    -68.5
                  ],
                  [
                    -73,
                    -68
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      }
    }
  }
})


In [ ]:
#interested bands
bands = ["HH", "angle"]
D28 = ee.Geometry.Rectangle([70.01, -69, 74, -67.5])
Image1 = S1A.first()
clipped_Image1 = Image1.clip(D28)
print(type(clipped_Image1))
# Image2 = ee.Image('COPERNICUS/S1_GRD/DD36')
my_map = folium.Map(location=[-68.0984, 74.0947], zoom_start=6)

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': "HH",
  'min': -30,
  'max': 0,}
my_map.add_ee_layer(clipped_Image1, vis_params, "image1")
my_map.add_ee_layer(Image1, vis_params, "image2")
my_map.add_child(folium.LayerControl())
display(my_map)

<class 'ee.image.Image'>


In [ ]:
feature = ee.FeatureCollection('projects/ee-jay/assets/Coastline_Antarctica_v02')
clipped_Image2 = clipped_Image1.clipToCollection(feature)

# final = clipped_Image1.mask(clipped_Image2)

my_map2 = folium.Map(location=[-68.0984, 74.0947], zoom_start=6)
my_map2.add_ee_layer(clipped_Image2, vis_params, "image")
# my_map2.add_ee_layer(final, vis_params, "image1")
my_map2.add_child(folium.LayerControl())
display(my_map2)

Applying SNIC Algorithm

In [ ]:
seeds = ee.Algorithms.Image.Segmentation.seedGrid(100)

#guassian kernel for softening the image
kernel = ee.Kernel.gaussian(3)
img1 = clipped_Image1.convolve(kernel)
img = img1.select(['HH', 'angle'], ['HH', 'angle'])

#SNIC image segmentation
snic = ee.Algorithms.Image.Segmentation.SNIC(img, 20, 5, 8, 256, seeds)\
        .select(["HH_mean", "angle_mean", "clusters"], ["HH", "angle", "clusters"])
print(type(snic))
my_map1 = folium.Map(location=[-68.0984, 72.0947], zoom_start=6)
vis_params = {
  'bands': "HH",
  'min': -30,
  'max': 0,}
my_map1.add_ee_layer(snic, vis_params, "image2")
my_map1.add_child(folium.LayerControl())
display(my_map1)

<class 'ee.image.Image'>


In [ ]:
vectorscale = 40
segments = snic.select("clusters").reduceToVectors(
            geometry = snic.geometry(),
            scale = vectorscale,
            geometryType = 'polygon',            
            maxPixels = 1e20
        )     
        
segimageA = segments.reduceToImage(properties = ['label'], reducer = ee.Reducer.first()).rename('id')    
# print(segments.size().getInfo())

ratio = snic.select("HH").divide(snic.select("angle"))
icebergs = ratio.gt(-0.2).selfMask()
show_iceberg = icebergs.clip(D28)  
print(type(icebergs))

my_map2 = folium.Map(location=[-68.0984, 74.0947], zoom_start=6)
vis_params = {
  'min': -10,
  'max': 0,}
my_map2.add_ee_layer(icebergs, vis_params, "image2")
my_map2.add_ee_layer(show_iceberg, vis_params, "image3")
my_map2.add_child(folium.LayerControl())
display(my_map2)

<class 'ee.image.Image'>


In [ ]:
# Convert iceberg raster to vector
vectors = show_iceberg.reduceToVectors(geometry = show_iceberg.geometry(),
            scale = vectorscale,
            geometryType = 'polygon',            
            maxPixels = 1e20)
# print(vectors.size().getInfo())

def addArea(feature):
  return feature.set({"area": feature.geometry().area(10), "centroid": feature.geometry().centroid(10)});

# Add area and centroid information to the iceberg vectors
areaAdded = vectors.map(addArea)
target = areaAdded
num = vectors.size().getInfo()
print(num)

4


In [ ]:
features = target.getInfo()['features']
print(len(features))
area = 0
for k in range(len(features)):
  print(features[k]['properties'])
# centroid = features[1]['properties']['centroid']['coordinates']
# print(centroid)

4
{'area': 5777537370.092439, 'centroid': {'type': 'Point', 'coordinates': [71.86683806084024, -68.83814865759291]}, 'count': 3635525, 'label': 1}
{'area': 1249487.4228872724, 'centroid': {'type': 'Point', 'coordinates': [70.54217000383113, -68.70115980913414]}, 'count': 786, 'label': 1}
{'area': 14826884.595294125, 'centroid': {'type': 'Point', 'coordinates': [70.96913836003495, -67.59207551312217]}, 'count': 9326, 'label': 1}
{'area': 1602506158.8435507, 'centroid': {'type': 'Point', 'coordinates': [70.9581971365636, -68.18864320507402]}, 'count': 1008054, 'label': 1}
